# This notebook creates a consolidated and curated 5-year world happiness data from year 2020 to 2024.

In [6]:
import pandas as pd
import requests
import os

# ================================ #
#           CONFIGURATION          #
# ================================ #

DATA_YEARS = range(2020, 2025)  # Years 2020-2024
RAW_FILES = [f"srs_file_{year}.csv" for year in DATA_YEARS]
STAGING_FILES = [f"stg_file_{year}.csv" for year in DATA_YEARS]
INTEGRATED_FILE = "int_happiness_dataset.csv"
REPORTING_FILE = "reporting_happiness_dataset.csv"
COUNTRY_TO_CONTINENT_FILE = "country_to_continent.csv"

# ================================ #
#           SOURCING DATA          #
# ================================ #

# Mapping of data sources
DATA_URLS = {
    2020: "https://happiness-report.s3.amazonaws.com/2020/WHR20_DataForFigure2.1.xls",
    2021: "https://happiness-report.s3.amazonaws.com/2021/DataForFigure2.1WHR2021C2.xls",
    2022: "https://happiness-report.s3.amazonaws.com/2022/Appendix_2_Data_for_Figure_2.1.xls",
    2023: "https://happiness-report.s3.amazonaws.com/2023/DataForFigure2.1WHR2023.xls",
    2024: "https://happiness-report.s3.amazonaws.com/2024/DataForFigure2.1+with+sub+bars+2024.xls"
}

def download_and_convert():
    """Download Excel files and convert to CSV."""
    for year, url in DATA_URLS.items():
        excel_file = f"srs_file_{year}.xls"
        csv_file = f"srs_file_{year}.csv"

        # Download file
        response = requests.get(url, stream=True)
        with open(excel_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"✅ Downloaded: {excel_file}")

        # Convert to CSV
        df = pd.read_excel(excel_file, engine="xlrd" if excel_file.endswith(".xls") else "openpyxl")
        df.to_csv(csv_file, index=False)
        print(f"✅ Converted to CSV: {csv_file}")

# ================================ #
#           STAGING DATA           #
# ================================ #

COLUMN_MAPPING = {
    'Country name': 'Country',
    'Regional indicator': 'Region',
    'Ladder score': 'Ladder Score',
    'Happiness score': 'Ladder Score',  # 2022 uses this
    'Logged GDP per capita': 'GDP per Capita',
    'Explained by: Log GDP per capita': 'GDP per Capita',  # 2024
    'Explained by: GDP per capita': 'GDP per Capita',  # 2022
    'Social support': 'Social Support',
    'Explained by: Social support': 'Social Support',  # 2022, 2024
    'Healthy life expectancy': 'Healthy Life Expectancy',
    'Explained by: Healthy life expectancy': 'Healthy Life Expectancy',
    'Freedom to make life choices': 'Freedom to Make Life Choices',
    'Explained by: Freedom to make life choices': 'Freedom to Make Life Choices',
    'Generosity': 'Generosity',
    'Explained by: Generosity': 'Generosity',
    'Perceptions of corruption': 'Perceptions of Corruption',
    'Explained by: Perceptions of corruption': 'Perceptions of Corruption',
    'Year': 'Year'
}

REQUIRED_COLUMNS = [
    'Year', 'Country', 'Region', 'Ladder Score', 'GDP per Capita', 
    'Social Support', 'Healthy Life Expectancy', 'Freedom to Make Life Choices', 
    'Generosity', 'Perceptions of Corruption'
]

def clean_dataframe(df):
    """Remove spaces, trailing asterisks, and standardize column names."""
    df = df.rename(columns=COLUMN_MAPPING)
    df = df[[col for col in REQUIRED_COLUMNS if col in df.columns]]
    df = df.dropna(subset=["Ladder Score", "GDP per Capita"])  
    # Strip leading/trailing spaces and trailing '*'
    str_cols = df.select_dtypes(include=["object"]).columns
    df[str_cols] = df[str_cols].apply(lambda col: col.str.strip().str.rstrip('*'))
    return df

# Process each dataset
for year, raw_file in zip(DATA_YEARS, RAW_FILES):
    if os.path.exists(raw_file):
        df = pd.read_csv(raw_file)
        df = clean_dataframe(df)
        df["Year"] = year
        df.to_csv(f"stg_file_{year}.csv", index=False)
        print(f"✅ Processed: stg_file_{year}.csv")

# ================================ #
#         INTEGRATION STEP         #
# ================================ #

# Load all staging datasets
dataframes = [pd.read_csv(file) for file in STAGING_FILES if os.path.exists(file)]
merged_df = pd.concat(dataframes, ignore_index=True)

# Load region/continent mapping
continent_region_mapping_df = pd.read_csv(COUNTRY_TO_CONTINENT_FILE)
country_to_region = dict(zip(continent_region_mapping_df["name"], continent_region_mapping_df["sub-region"]))
region_to_continent = dict(zip(continent_region_mapping_df["sub-region"], continent_region_mapping_df["region"]))

# Fill in missing "Region" and "Continent"
merged_df["Region"] = merged_df["Country"].map(country_to_region).fillna("Unknown")
merged_df["Continent"] = merged_df["Region"].map(region_to_continent).fillna("Unknown")

# Manual overrides for missing data
manual_updates = {
    "Bosnia and Herzegovina": {"Region": "Southern Europe", "Continent": "Europe"},
    "Congo (Brazzaville)": {"Region": "Sub-Saharan Africa", "Continent": "Africa"},
    "Hong Kong S.A.R. of China": {"Region": "Eastern Asia", "Continent": "Asia"},
    "Kosovo": {"Region": "Southern Europe", "Continent": "Europe"},
    "North Cyprus": {"Region": "Western Asia", "Continent": "Asia"},
    "North Macedonia": {"Region": "Southern Europe", "Continent": "Europe"}
}
for country, values in manual_updates.items():
    merged_df.loc[merged_df["Country"] == country, ["Region", "Continent"]] = values["Region"], values["Continent"]

# Save final integrated dataset
merged_df.to_csv(INTEGRATED_FILE, index=False)
print(f"✅ Integrated dataset saved as {INTEGRATED_FILE}")

# ================================ #
#    ANALYSIS & REPORTING STEP     #
# ================================ #

# Calculate average regional Ladder Score and GDP per Capita
regional_avg = merged_df.groupby(['Year', 'Region'])[['Ladder Score', 'GDP per Capita', 'Social Support', 
                                                      'Healthy Life Expectancy', 'Freedom to Make Life Choices', 
                                                      'Generosity', 'Perceptions of Corruption']].mean().reset_index()

# Rename columns to match requirements
regional_avg.rename(columns={
    'Ladder Score': 'Average Regional Ladder Score',
    'GDP per Capita': 'Average Regional GDP per Capita',
    'Social Support': 'Average Regional Social Support',
    'Healthy Life Expectancy': 'Average Regional Healthy Life Expectancy', 
    'Freedom to Make Life Choices': 'Average Regional Freedom to Make Life Choices', 
    'Generosity': 'Average Regional Generosity', 
    'Perceptions of Corruption': 'Average Regional Perceptions of Corruption'
}, inplace=True)

# Merge the calculated averages back into the original dataset
merged_df = merged_df.merge(regional_avg, on=['Year', 'Region'], how='left')

# Ensure correct column order
merged_df = merged_df[['Year', 'Country', 'Region', 'Continent', 
                       'Ladder Score', 'Average Regional Ladder Score',
                       'GDP per Capita', 'Average Regional GDP per Capita',
                       'Social Support', 'Average Regional Social Support',
                       'Healthy Life Expectancy', 'Average Regional Healthy Life Expectancy',
                       'Freedom to Make Life Choices', 'Average Regional Freedom to Make Life Choices', 
                       'Generosity', 'Average Regional Generosity',
                       'Perceptions of Corruption', 'Average Regional Perceptions of Corruption']]

# Save reporting dataset
merged_df.to_csv(REPORTING_FILE, index=False)
print(f"✅ Reporting dataset saved as {REPORTING_FILE}")


✅ Processed: stg_file_2020.csv
✅ Processed: stg_file_2021.csv
✅ Processed: stg_file_2022.csv
✅ Processed: stg_file_2023.csv
✅ Processed: stg_file_2024.csv
✅ Integrated dataset saved as int_happiness_dataset.csv
✅ Reporting dataset saved as reporting_happiness_dataset.csv
